In [12]:
import os
from src import utils 
import pandas as pd
import numpy as np

def translate_yaml(yaml_path):
    yaml_data = utils.load_yaml(yaml_path)

    dataset_config = yaml_data.pop('dataset_config')
    model_config = yaml_data.pop('model_config')
    # optim_config = yaml_data.pop('optim_config')
    other_config = yaml_data.pop('other_config')

    dataset_config.pop('data_paths')

    yaml_data['data_augmentation'] = ''.join([e[0] for e in yaml_data['data_augmentation']])
    yaml_data['input_data_shape'] = '_'.join([str(e) for e in yaml_data['input_data_shape']])
    yaml_data['output_data_shape'] = '_'.join([str(e) for e in yaml_data['output_data_shape']])
    yaml_data['test_metric_indexes'] = '_'.join([str(e) for e in yaml_data['test_metric_indexes']])

    yaml_data.update(dataset_config)
    yaml_data.update(model_config)
    # yaml_data.update(optim_config)
    yaml_data.update(other_config)

    return yaml_data

def load_test_results(test_metrics_path):
    metric_dict= {}
    for metric in os.listdir(test_metrics_path):
        metric_name = metric[:-4]
        metric_data = np.load(os.path.join(test_metrics_path, metric))
        metric_dict[f'{metric_name}_mean'] = np.mean(metric_data) if len(metric_data) > 0 else 0
        metric_dict[f'{metric_name}_std'] = np.std(metric_data) if len(metric_data) > 0 else 0
    return metric_dict
        

In [13]:
results_path = './results'

if not os.path.exists(results_path):
    raise ValueError('There is no results folder.')

dataset_list = os.listdir(results_path)
dataset_path_list = [os.path.join(results_path, e) for e in os.listdir(results_path)]

all_results = {}

for dataset, dataset_path in zip(dataset_list, dataset_path_list):
    model_list = os.listdir(dataset_path)
    model_path_list = [os.path.join(dataset_path, e) for e in os.listdir(dataset_path)]
    for model, model_path in zip(model_list, model_path_list):
        additfolder_list = os.listdir(model_path)
        additfolder_path_list = [os.path.join(model_path, e) for e in os.listdir(model_path)]
        cont = 0
        for additfolder, additfolder_path in zip(additfolder_list, additfolder_path_list):
            config_list = os.listdir(additfolder_path)
            config_path_list = [os.path.join(additfolder_path, e) for e in os.listdir(additfolder_path)]
            for config, config_path in zip(config_list, config_path_list):
                yaml_path = os.path.join(config_path, 'train_configuration.yaml')
                test_metrics_path = os.path.join(config_path, 'test_metrics')
                predicted_images_path = os.path.join(config_path, 'predicted_images')
                
                yaml_data = translate_yaml(yaml_path)
                yaml_data.update(load_test_results(test_metrics_path))
                all_results[f'{model}_{dataset}_{cont}'] = yaml_data

                cont += 1


In [14]:
df = pd.DataFrame(all_results).T
nunique = df.nunique()
cols_to_drop = nunique[nunique == 1].index
df.drop(cols_to_drop, axis=1)

,input_data_shape,learning_rate,number_of_epochs,output_data_shape,ssim_mean,psnr_mean,mse_mean
unet_EM_0,90_64_64_1,0.0005,20,90_256_256_1,0.42652,14.290348,0.037236
unet_EM_1,90_64_64_1,0.001,5,90_256_256_1,0.40454,15.072568,0.031099
unet_EM_2,90_64_64_1,0.005,20,90_256_256_1,0.414133,13.703012,0.042628
unet_EM_3,90_64_64_1,0.001,10,90_256_256_1,0.423665,14.376402,0.036506
unet_EM_4,90_64_64_1,0.0005,10,90_256_256_1,0.427813,14.359768,0.036646
unet_EM_5,90_64_64_1,0.0005,5,90_256_256_1,0.399906,14.336056,0.036846
unet_EM_6,90_64_64_1,0.005,5,90_256_256_1,0.312463,14.393086,0.036366
unet_EM_7,90_64_64_1,0.005,10,90_256_256_1,0.435949,15.348394,0.029185
unet_EM_8,90_64_64_1,0.001,20,90_256_256_1,0.421442,14.149803,0.038461
unet_MitoTracker_small_0,1440_64_64_1,0.0005,20,1440_256_256_1,0.491023,15.514202,0.028092
